In [1]:
flag = False

In [16]:
if flag:
    print(Run)
flag = True

canons                   5.0
holdout                 10.0
random_state          1522.0
dense_layer_length      10.0
accuracy_sop             0.4
accuracy_dnn             0.6
dtype: float64


In [3]:
train   = 200
holdout = 10
D       = 10
canons  = 5
random_state = 1
dense_layer_length = 10
table = 'figures'
if_exists = 'append'

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

# Verify the shapes of the loaded data
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Testing images shape:", test_images.shape)
print("Testing labels shape:", test_labels.shape)

# Normalize the pixel values (0-255 to 0-1)
train_images = train_images / 255.0
test_images = test_images / 255.0

2025-04-23 05:51:23.848814: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 05:51:23.859063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745405483.871191    6139 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745405483.874793    6139 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 05:51:23.887058: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Training images shape: (60000, 28, 28)
Training labels shape: (60000,)
Testing images shape: (10000, 28, 28)
Testing labels shape: (10000,)


In [5]:
indices = pd.DataFrame(np.reshape(train_images,(60000,-1))).mean().sort_values(ascending = False)[:D].index

X_train = np.reshape(train_images,(60000,-1))[:train,indices]
X_test  = np.reshape(test_images,(10000,-1))[:holdout,indices]
y_train = train_labels[:train]
y_test  = test_labels[:holdout]

In [6]:

len(X_train),len(y_test)

(200, 10)

In [7]:
from os import chdir
#chdir('..')
import sopy as sp

from connect import conn
engine = conn()

In [8]:
Run = pd.Series()

d = pd.DataFrame(X_train, columns = range(1,D+1)).describe()
ir= pd.DataFrame(X_train, columns = range(1,D+1))
cl= pd.Series(y_train)


sigmas   =  [ [ 0.63 for c in d ] for cl1 in cl.unique() ]


spacings  = [ d[c]['std']/4 for c in d]
lattices = [ np.arange( 0,1, spacings[dd] ) for dd,c in enumerate(d) ]

Run['canons'] = canons
Run['holdout'] = holdout
Run['random_state'] = random_state
Run['dense_layer_length'] = dense_layer_length

d = pd.DataFrame(X_train, columns = range(1,D+1)).describe()
ir= pd.DataFrame(X_train, columns = range(1,D+1))
cl= pd.Series(y_train)

In [9]:
F = {}
for f in range(10):
    u = sp.vector()
    for index in cl[cl==f].index:
        u += sp.vector().gaussian(a=1, ls = D*[0], positions = ir.loc[index].values, sigmas = sigmas[f] , lattices = lattices)
    F[f] = u.mul(1./u.n()).fibonacci(partition = canons )
    
    print(len(u))

2025-04-23 05:51:27.926271: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


21
26
20
21
21
13
19
21
15
23


In [10]:
[[ F[i].dot(F[j]) for i in range(j+1,10)] for j in range(10)]

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.5005998468163152>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.8219749799250786>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5225195138887204>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.2574739435701595>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.6788105774431501>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.48387789329753866>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5329454803066987>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.4352942207439513>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5705535058874565>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.7768075116211924>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.7150829500513055>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.7071264290644605>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.7917690278073853>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.7375253939429394>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.679841995207694>,
  <tf.Ten

In [11]:
y_flat_pred = [ [([ v.dot(sp.vector().delta(a= 1/np.sqrt(np.prod(spacings)), positions = x, spacings = spacings , lattices = lattices )) for v in list(F.values()) ])]for x in X_test]

In [12]:
y_pred=[np.argmax(y) for y in y_flat_pred]
np.mean( [ max(y) for y in y_flat_pred] )

np.float64(0.5742094670446221)

In [13]:
accuracy_sop = accuracy_score(y_test, y_pred)
Run['accuracy_sop'] = accuracy_sop

In [14]:
##GEMINI GENERATED THIS CODE

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Define the Single Layer Dense Neural Network (Softmax Regression)
class SingleLayerNN:
    def __init__(self, input_size, output_size, learning_rate=0.01, epochs=1000):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = np.random.rand(input_size, output_size) - 0.5  # Initialize weights randomly
        self.bias = np.zeros((1, output_size))  # Initialize bias to zeros

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def forward(self, X):
        self.z = np.dot(X, self.weights) + self.bias
        self.y_hat = self.softmax(self.z)
        return self.y_hat

    def backward(self, X, y, y_hat):
        m = X.shape[0]
        dz = y_hat - self.one_hot_encode(y)
        dw = (1/m) * np.dot(X.T, dz)
        db = (1/m) * np.sum(dz, axis=0, keepdims=True)
        return dw, db

    def one_hot_encode(self, y):
        one_hot = np.zeros((len(y), self.output_size))
        one_hot[np.arange(len(y)), y] = 1
        return one_hot

    def update_parameters(self, dw, db):
        self.weights -= self.learning_rate * dw
        self.bias -= self.learning_rate * db

    def train(self, X_train, y_train):
        for epoch in range(self.epochs):
            y_hat = self.forward(X_train)
            dw, db = self.backward(X_train, y_train, y_hat)
            self.update_parameters(dw, db)
            if (epoch + 1) % 100 == 0:
                loss = self.categorical_cross_entropy(self.one_hot_encode(y_train), y_hat)
                #print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}")

    def predict(self, X_test):
        y_prob = self.forward(X_test)
        y_pred = np.argmax(y_prob, axis=1)
        return y_pred

    def categorical_cross_entropy(self, y_true, y_pred):
        # Add a small epsilon to avoid log(0)
        epsilon = 1e-15
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(np.sum(y_true * np.log(y_pred_clipped), axis=1))
        return loss

# 5. Initialize and train the neural network
input_size = X_train_scaled.shape[1]  # Number of features (4)
output_size = len(np.unique(y_train))  # Number of classes (3)
learning_rate = 0.1
epochs = 2000

model = SingleLayerNN(input_size, output_size, learning_rate, epochs)
model.train(X_train_scaled, y_train)

# 6. Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# 7. Evaluate the model
accuracy_snn = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy_snn:.4f}")

# You can also print the classification report for more detailed metrics
#from sklearn.metrics import classification_report
#print("\nClassification Report:")
#print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Test Accuracy: 0.6000


In [15]:
Run['accuracy_dnn'] = accuracy_snn
Run#.to_sql(table,engine,if_exists = if_exists)

canons                   5.0
holdout                 10.0
random_state          1522.0
dense_layer_length      10.0
accuracy_sop             0.4
accuracy_dnn             0.6
dtype: float64